In [1]:
# import library
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Get Level 40 Stats Table ( lvl40_stats_df )

In [2]:
# Request to website and download HTML contents
lvl40_stats_url = 'https://feheroes.fandom.com/wiki/Level_40_stats_table'
lvl40_stats_req = requests.get(lvl40_stats_url)
lvl40_stats_content = lvl40_stats_req.text

In [3]:
# Make soup pretty
lvl40_stats_soup=BeautifulSoup(lvl40_stats_content, 'html.parser')

In [4]:
# Scrape lvl40_stats headers
headers = []
for i in lvl40_stats_soup.findAll('th'):
    if i.text == "Weapon":
        headers.append("Weapon_Type")
    else:
        headers.append(i.text)

# DataFrame lvl40_stats
lvl40_stats_df = pd.DataFrame(columns=headers)

In [5]:
# Find all elements in html script with character info
characters = lvl40_stats_soup.findAll('tr', class_='hero-filter-element')
row = 0

# Iterate through elements to isolate relevant data and add it to dataframe
for character in characters:
    lines = character.findAll('td')
    char_info = []
    for line in lines:
        try:
            char_info.append(line.select('img')[0].get('alt'))
        except:
            char_info.append(line.text)
        
    # Add character stats and info to lvl40_stats_df
    lvl40_stats_df.loc[row] = char_info
    row += 1

In [6]:
# Drop irrelevant column
lvl40_stats_df = lvl40_stats_df.drop([''], axis=1)

In [7]:
# View scraped dataframe (lvl40_stats_df)
lvl40_stats_df

,Hero,Entry,Move,Weapon_Type,HP,Atk,Spd,Def,Res,Total
0,Abel: The Panther,Shadow Dragon / (New) Mystery,Cavalry,Blue Lance,39,33,32,25,25,154
1,Aelfric: Custodian Monk,Three Houses,Infantry,Red Tome,43,40,31,20,37,171
2,Alfonse: Askran Duo,Heroes,Infantry,Blue Tome,41,38,17,35,30,161
3,Alfonse: Prince of Askr,Heroes,Infantry,Red Sword,43,35,25,32,22,157
4,Alfonse: Spring Prince,Heroes,Cavalry,Green Axe,41,35,33,30,18,157
...,...,...,...,...,...,...,...,...,...,...
787,Zelgius: Jet-Black General,Radiant Dawn,Armored,Red Sword,46,36,33,38,21,174
788,Zelot: Avowed Groom,The Binding Blade,Cavalry,Green Axe,41,37,25,39,25,167
789,Zephiel: The Liberator,The Binding Blade,Armored,Red Sword,55,35,16,38,24,168
790,Zephiel: Winter's Crown,The Blazing Blade,Armored,Red Sword,48,38,19,38,36,179


In [25]:
# Split Hero column into Name and Epithet
name_split_df = lvl40_stats_df['Hero'].str.split(': ', expand=True).rename(columns = {0:'Name', 1:'Epithet'})

#Split weapon column into Color and Generic Weapon Type
weapon_split_df = lvl40_stats_df['Weapon_Type'].str.split(' ', expand=True).rename(columns = {0:'Color', 1:'Weapon_Type_Generic'})

# Concatenate new columns on lvl40_stats_df
lvl40_stats_df = pd.concat([lvl40_stats_df, name_split_df, weapon_split_df], axis=1)

In [27]:
# View final dataframe
lvl40_stats_df

,Hero,Entry,Move,Weapon_Type,HP,Atk,Spd,Def,Res,Total,Name,Epithet,Color,Weapon_Type_Generic
0,Abel: The Panther,Shadow Dragon / (New) Mystery,Cavalry,Blue Lance,39,33,32,25,25,154,Abel,The Panther,Blue,Lance
1,Aelfric: Custodian Monk,Three Houses,Infantry,Red Tome,43,40,31,20,37,171,Aelfric,Custodian Monk,Red,Tome
2,Alfonse: Askran Duo,Heroes,Infantry,Blue Tome,41,38,17,35,30,161,Alfonse,Askran Duo,Blue,Tome
3,Alfonse: Prince of Askr,Heroes,Infantry,Red Sword,43,35,25,32,22,157,Alfonse,Prince of Askr,Red,Sword
4,Alfonse: Spring Prince,Heroes,Cavalry,Green Axe,41,35,33,30,18,157,Alfonse,Spring Prince,Green,Axe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,Zelgius: Jet-Black General,Radiant Dawn,Armored,Red Sword,46,36,33,38,21,174,Zelgius,Jet-Black General,Red,Sword
788,Zelot: Avowed Groom,The Binding Blade,Cavalry,Green Axe,41,37,25,39,25,167,Zelot,Avowed Groom,Green,Axe
789,Zephiel: The Liberator,The Binding Blade,Armored,Red Sword,55,35,16,38,24,168,Zephiel,The Liberator,Red,Sword
790,Zephiel: Winter's Crown,The Blazing Blade,Armored,Red Sword,48,38,19,38,36,179,Zephiel,Winter's Crown,Red,Sword


In [26]:
pd.DataFrame(lvl40_stats_df['Color'].value_counts())

,Color
Red,226
Blue,212
Green,187
Colorless,167


## Get Hero Skills Table

In [10]:
# Request to website and download HTML contents
hero_skills_url = 'https://feheroes.fandom.com/wiki/Hero_skills_table'
hero_skills_req = requests.get(hero_skills_url)
hero_skills_content = hero_skills_req.text

# Make soup pretty
hero_skills_soup=BeautifulSoup(hero_skills_content, 'html.parser')

In [11]:
# Scrape hero_skills headers
headers = []
string = "Type"
for i in hero_skills_soup.findAll('th'):
    if i.text == "Weapon":
        headers.append("Weapon_" + string)
        string = "Skill"
    else:
        headers.append(i.text)

# DataFrame lvl40_stats
hero_skills_df = pd.DataFrame(columns=headers)

In [13]:
# Find all elements in html script with character info
characters = hero_skills_soup.findAll('tr', class_='hero-filter-element')
row = 0

# Iterate through elements to isolate relevant data and add it to dataframe
for character in characters:
    lines = character.findAll('td')
    char_info = []
    for line in lines:
        try:
            char_info.append(line.select('img')[0].get('alt'))
        except:
            char_info.append(line.text)
        
    # Add character stats and info to lvl40_stats_df
    hero_skills_df.loc[row] = char_info
    row += 1

In [14]:
# Drop irrelevant column
hero_skills_df = hero_skills_df.drop([''], axis=1)

In [28]:
# View scraped dataframe (hero_skills_df)
hero_skills_df

,Hero,Move,Weapon_Type,Weapon_Skill,Assist,Special,Passive A,Passive B,Passive C
0,Abel: The Panther,Cavalry,Blue Lance,Brave Lance+Panther Lance,—,Aegis,HP +5,Swordbreaker 3,—
1,Aelfric: Custodian Monk,Infantry,Red Tome,Rauðrrabbit+,—,Glacies,Atk/Res Push 3,—,Threat. Def/Res 2
2,Alfonse: Askran Duo,Infantry,Blue Tome,Imbued Koma,—,Open the Future,Close Counter,Special Spiral 3,Def/Res Oath 3
3,Alfonse: Prince of Askr,Infantry,Red Sword,Fólkvangr,—,Sol,Death Blow 3,—,Spur Atk 3
4,Alfonse: Spring Prince,Cavalry,Green Axe,Giant Spoon+,—,Noontime,Sturdy Blow 2,—,Def Smoke 3
...,...,...,...,...,...,...,...,...,...
787,Zelgius: Jet-Black General,Armored,Red Sword,Alondite,—,Black Luna,Fierce Stance 3,Warp Powder,Panic Ploy 3
788,Zelot: Avowed Groom,Cavalry,Green Axe,Love Candelabra+,—,Ignis,Atk/Def Form 3,—,Drive Atk 2
789,Zephiel: The Liberator,Armored,Red Sword,Eckesachs,—,Reprisal,Life and Death 3,Wary Fighter 3,—
790,Zephiel: Winter's Crown,Armored,Red Sword,Bellringer+,—,Glacies,—,Vengeful Fighter 3,Def/Res Gap 3


In [29]:
# pd.DataFrame(hero_skills_df['Weapon_Skill'].value_counts()).head(11)